In [1]:
from BPM_MF_algo_numba_class import BPM_MatrixFactorization, fit
import pandas as pd
import numpy as np
from ypstruct import structure
from scipy.sparse import coo_matrix
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import json

In [ ]:
#Import data
colnames = ['userId', 'movieId', 'rating', 'timeStamp']
data_df = pd.read_csv('ml-100k/u.data',sep='\t', 
                        names = colnames, header = None)
# data_df = pd.read_csv('E:/DW/확률그래프모델/project/ml-25m/ratings.csv')
#rating_df = data_df.pivot(index='userId', columns='movieId', values='rating')

userId_list = Counter(data_df['userId'])
movieId_list = Counter(data_df['movieId'])
ratings_list = Counter(data_df['rating'])

userId_arr  = data_df['userId'].values.copy()
movieId_arr = data_df['movieId'].values.copy()
rating_arr = data_df['rating'].values.copy()

#coordinate transformation minus 1
userId_arr -= 1
movieId_arr -= 1

#problem setting
problem = structure()
problem.maxiter = 250
problem.rows = max(userId_arr)+1
problem.cols = max(movieId_arr)+1


X_train, X_test = train_test_split(data_df, test_size=0.2, random_state=42)

#Cross validation setting
cv = KFold(5, shuffle=True, random_state=42)
cv_output = []

In [ ]:
for k in range(5) :
    #rmse
    temp_list = []
    for i, (idx_train, idx_validation) in enumerate(cv.split(X_train)):
        print("{}_th cv computing".format(i))
        df_train = data_df.iloc[idx_train]
        df_validation = data_df.iloc[idx_validation]

        userId_tr  = df_train['userId'].values
        movieId_tr = df_train['movieId'].values
        rating_tr = df_train['rating'].values

        userId_vd  = df_validation['userId'].values
        movieId_vd = df_validation['movieId'].values
        rating_vd = df_validation['rating'].values

        userId_tr -= 1
        movieId_tr -= 1

        userId_vd -= 1
        movieId_vd -= 1

        sparse_train_m = coo_matrix((rating_tr, (userId_tr, movieId_tr)), \
                        shape=(problem.rows, problem.cols))

        sparse_validation_m = coo_matrix((rating_vd, (userId_vd, movieId_vd)), \
                        shape=(problem.rows, problem.cols))

        problem.data_m = sparse_train_m
        problem.test_m = sparse_validation_m

        #Parameters setting
        params = structure()
        params.alpha = 0.2
        params.beta = 5
        params.R = 4
        params.normal_loc = 0
        params.normal_var = 0.5
        params.latent_k = 5*(k+1)

        #run algorithm
        temp_list.append(fit(problem,params))

        #print result
        print("{}_th latent ".format(params.latent_k), "cv_{}_th MAE :".format(i), temp_list[i].MAE)
        print("{}_th latent ".format(params.latent_k), "cv_{}_th CMAE :".format(i), temp_list[i].CMAE)
        print("{}_th latent ".format(params.latent_k), "cv_{}_th 0_1_loss :".format(i), temp_list[i].zero_one_loss)

---

In [2]:
#Import data
colnames = ['userId', 'movieId', 'rating', 'timeStamp']
data_df = pd.read_csv('ml-100k/u.data',sep='\t', 
                        names = colnames, header = None)

In [3]:
data_df

,userId,movieId,rating,timeStamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
userId_list = Counter(data_df['userId'])
movieId_list = Counter(data_df['movieId'])
ratings_list = Counter(data_df['rating'])

userId_arr  = data_df['userId'].values.copy()
movieId_arr = data_df['movieId'].values.copy()
rating_arr = data_df['rating'].values.copy()

In [5]:
#coordinate transformation minus 1
userId_arr -= 1
movieId_arr -= 1

#problem setting
problem = structure()
problem.maxiter = 250
problem.rows = max(userId_arr)+1
problem.cols = max(movieId_arr)+1

In [6]:
X_train, X_test = train_test_split(data_df, test_size=0.2, random_state=42)

In [7]:
idx_train = X_train.index
idx_test = X_test.index

In [8]:
userId_train = X_train['userId'].values - 1
movieId_train = X_train['movieId'].values - 1
rating_train = X_train['rating'].values

sparse_train_matrix = coo_matrix((rating_train, (userId_train, movieId_train)), \
                           shape=(problem.rows, problem.cols))

In [9]:
userId_test = X_test['userId'].values - 1
movieId_test = X_test['movieId'].values - 1
rating_test = X_test['rating'].values

sparse_test_matrix = coo_matrix((rating_test, (userId_test, movieId_test)), \
                           shape=(problem.rows, problem.cols))

In [10]:
problem.data_m = sparse_train_matrix
problem.test_m = sparse_test_matrix

In [11]:
# parameters setting
params = structure()
params.alpha = 0.2
params.beta = 5
params.R = 4
params.normal_loc = 0
params.normal_var = 0.5
params.latent_k = 5

In [14]:
problem.data_m.row

array([806, 473, 462, ..., 436, 283, 221], dtype=int32)

In [12]:
factorization = BPM_MatrixFactorization(problem, params)

AttributeError: 'NoneType' object has no attribute '_code'

In [ ]:
factorization.normal_var

In [ ]:
add(1, 2)

In [ ]:
import numpy as np
from numba import int32, float32    # import the types
from numba.experimental import jitclass

spec = [
    ('value', int32),               # a simple scalar field
    ('array', float32[:]),          # an array field
]

@jitclass(spec)
class Bag(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)

    #@property
    def size(self):
        return self.array.size

    def increment(self, val):
        for i in range(self.size):
            self.array[i] += val
        return self.array

    def add(x, y):
        return x + y

n = 21
mybag = Bag(n)

In [ ]:
mybag.size